# Model 4: Batch Size

This notebook contains the construction and training of the model iterations and various experiments. The notebook is split up into four sections: training mode selection (where the model will run), set-up, model constrution, and training. 

Evaluation will take place in the *model_optimization_and_evaluation.ipynb* notebook found in the *notebooks* folder.

In [1]:
# importing necessary packages and libraries
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import optimizers
from keras import applications 
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import pickle

Using TensorFlow backend.


## Step 1: Training Mode Selection

In the cell below, specify the training mode for the model. This will determine the location from which the source data is drawn, and to which the trained models (and training histories) are saved. 

- **training_mode = 'floydhub'** (runs on Floydhub)
- **training_mode = 'local'** (runs on local disk and processor)

In [2]:
# select training mode
training_mode = 'floydhub'

## Step 2: Set-up

In [3]:
# directory base paths
data_path_local = '../../data/0002_array_data/train_data/'
model_path_local = '../../notebooks/model_construction/saved_models/'
data_path_floydhub = '/floyd/input/capstone_mushrooms/'
model_path_floydhub = '/floyd/home/'

# setting directory paths based on training mode selection
if training_mode == 'floydhub':
    data_path = data_path_floydhub
    model_path = model_path_floydhub
elif training_mode == 'local':
    data_path = data_path_local
    model_path = model_path_local
else:
    raise Exception('Please choose valid training mode: "floydhub" or "local".')

In [4]:
# loading the training and validation data subsets
X_train = np.load(f'{data_path}X_train_data.npy')
y_train = np.load(f'{data_path}y_train_data.npy')
X_val = np.load(f'{data_path}X_val_data.npy')
y_val = np.load(f'{data_path}y_val_data.npy')

In [5]:
# setting training parameters
# batch_size = 32
n_classes = 20
n_epochs = 15
img_shape = X_train.shape[1:]
model_names = []
model_list = []
model_hists = []

## Step 3: Model Construction

### Trial 1 - 8

In [6]:
# defining the model architecture
m4_t1 = models.Sequential()

# convolution/max pool stacks
m4_t1.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=img_shape, padding='same'))
m4_t1.add(layers.MaxPooling2D((2,2)))

m4_t1.add(layers.Conv2D(64,(3,3), activation='relu', padding='same'))
m4_t1.add(layers.MaxPooling2D((2,2)))

m4_t1.add(layers.Conv2D(128,(3,3), activation='relu', padding='same'))
m4_t1.add(layers.MaxPooling2D((2,2)))

m4_t1.add(layers.Conv2D(256,(3,3), activation='relu', padding='same'))
m4_t1.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m4_t1.add(layers.Flatten())
m4_t1.add(layers.Dense(512, activation='relu'))
m4_t1.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m4_t1.summary()
model_names.append('m4_t1')
model_list.append(m4_t1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 25, 256)      

### Trial 2 - 128

In [7]:
# defining the model architecture
m4_t2 = models.Sequential()

# convolution/max pool stacks
m4_t2.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=img_shape, padding='same'))
m4_t2.add(layers.MaxPooling2D((2,2)))

m4_t2.add(layers.Conv2D(64,(3,3), activation='relu', padding='same'))
m4_t2.add(layers.MaxPooling2D((2,2)))

m4_t2.add(layers.Conv2D(128,(3,3), activation='relu', padding='same'))
m4_t2.add(layers.MaxPooling2D((2,2)))

m4_t2.add(layers.Conv2D(256,(3,3), activation='relu', padding='same'))
m4_t2.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m4_t2.add(layers.Flatten())
m4_t2.add(layers.Dense(512, activation='relu'))
m4_t2.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m4_t2.summary()
model_names.append('m4_t2')
model_list.append(m4_t2)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 25, 25, 256)      

## Step 4: Training

In [8]:
# setting up standardization and augmentation parameters
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   fill_mode='nearest',
                                   horizontal_flip=True,
                                   vertical_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
%%time
# data standardization and augmentation
train_generator_1 = train_datagen.flow(X_train, y_train, batch_size=8)
val_generator_1 = val_datagen.flow(X_val, y_val, batch_size=8)
train_generator_2 = train_datagen.flow(X_train, y_train, batch_size=128)
val_generator_2 = val_datagen.flow(X_val, y_val, batch_size=128)

CPU times: user 880 ms, sys: 2.56 s, total: 3.44 s
Wall time: 3.44 s


In [10]:
# compiling loss functions
m4_t1.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['acc'])
m4_t2.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['acc'])

In [11]:
%%time
# setting up model saving checkpoints
m4_t1_cp = ModelCheckpoint(filepath=f'{model_path}m4_t1.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m4_t1_history = m4_t1.fit(train_generator_1,
                                steps_per_epoch=len(X_train)//8,
                                epochs=n_epochs,
                                callbacks=[m4_t1_cp],
                                validation_data=val_generator_1,
                                validation_steps=len(X_val)//8)

# adding training history to list
model_hists.append(m4_t1_history)

Epoch 1/15
1013/1013 [==============================] - 150s 148ms/step - loss: 2.7123 - acc: 0.1410 - val_loss: 2.6837 - val_acc: 0.2562

Epoch 00001: val_acc improved from -inf to 0.25619, saving model to /floyd/home/m4_t1.h5
Epoch 2/15
1013/1013 [==============================] - 81s 80ms/step - loss: 2.3319 - acc: 0.2485 - val_loss: 2.6121 - val_acc: 0.3192

Epoch 00002: val_acc improved from 0.25619 to 0.31920, saving model to /floyd/home/m4_t1.h5
Epoch 3/15
1013/1013 [==============================] - 81s 80ms/step - loss: 2.2204 - acc: 0.2794 - val_loss: 2.5052 - val_acc: 0.3416

Epoch 00003: val_acc improved from 0.31920 to 0.34165, saving model to /floyd/home/m4_t1.h5
Epoch 4/15
1013/1013 [==============================] - 81s 80ms/step - loss: 2.0865 - acc: 0.3320 - val_loss: 1.7294 - val_acc: 0.3853

Epoch 00004: val_acc improved from 0.34165 to 0.38529, saving model to /floyd/home/m4_t1.h5
Epoch 5/15
1013/1013 [==============================] - 82s 81ms/step - loss: 2.0017 

In [12]:
%%time
# setting up model saving checkpoints
m4_t2_cp = ModelCheckpoint(filepath=f'{model_path}m4_t2.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m4_t2_history = m4_t2.fit(train_generator_2,
                                steps_per_epoch=len(X_train)//128,
                                epochs=n_epochs,
                                callbacks=[m4_t2_cp],
                                validation_data=val_generator_2,
                                validation_steps=len(X_val)//128)

# adding training history to list
model_hists.append(m4_t2_history)

Epoch 1/15
63/63 [==============================] - 75s 1s/step - loss: 2.9776 - acc: 0.0962 - val_loss: 2.7815 - val_acc: 0.1758

Epoch 00001: val_acc improved from -inf to 0.17578, saving model to /floyd/home/m4_t2.h5
Epoch 2/15
63/63 [==============================] - 74s 1s/step - loss: 2.6385 - acc: 0.1629 - val_loss: 2.3672 - val_acc: 0.2185

Epoch 00002: val_acc improved from 0.17578 to 0.21848, saving model to /floyd/home/m4_t2.h5
Epoch 3/15
63/63 [==============================] - 73s 1s/step - loss: 2.4227 - acc: 0.2247 - val_loss: 2.1696 - val_acc: 0.2845

Epoch 00003: val_acc improved from 0.21848 to 0.28446, saving model to /floyd/home/m4_t2.h5
Epoch 4/15
63/63 [==============================] - 73s 1s/step - loss: 2.2881 - acc: 0.2708 - val_loss: 1.9673 - val_acc: 0.3416

Epoch 00004: val_acc improved from 0.28446 to 0.34164, saving model to /floyd/home/m4_t2.h5
Epoch 5/15
63/63 [==============================] - 74s 1s/step - loss: 2.1891 - acc: 0.2915 - val_loss: 1.8301

In [13]:
# creating dictionary for model names, models, and histories from respective lists
models_dict = {i:[j,k] for i,j,k in zip(model_names,model_list,model_hists)}

In [16]:
(val_loss_1, val_accuracy_1) = m4_t1.evaluate(val_generator_1,verbose=1)
print(f'{key} Val Accuracy 1: {round((val_accuracy_1*100),2)}%')
print(f'{key} Val Loss 1: {round(val_loss_1,4)}')
print('---')
(val_loss_2, val_accuracy_2) = m4_t2.evaluate(val_generator_2,verbose=1)
print(f'{key} Val Accuracy 2: {round((val_accuracy_2*100),2)}%')
print(f'{key} Val Loss 2: {round(val_loss_2,4)}')
print('---')

102/102 [==============================] - 2s 15ms/step
m4_t1 Val Accuracy 1: 54.94%
m4_t1 Val Loss 1: 0.1016
---
7/7 [==============================] - 1s 180ms/step
m4_t1 Val Accuracy 2: 49.26%
m4_t1 Val Loss 2: 1.2017
---


In [17]:
# saving training histories
for key, value in models_dict.items():
    with open(f'{model_path}{key}_history', 'wb') as file_pi:
        pickle.dump(value[1].history, file_pi)
    print(f'{key}_history saved in {model_path}')

m4_t1_history saved in /floyd/home/
m4_t2_history saved in /floyd/home/
